In [7]:
%pip install tensorflow gradio numpy

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/20/ba/7560053dbd0b2f5e3654c6d0ad57267558d2d17aec6604a9f6a3fdfb4d5c/tensorflow-2.15.0-cp310-cp310-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/77/be/18e65093f8841b6ebaf09bc1539735a6fcb09a17d202d03d040387eb95cc/gradio-4.7.1-py3-none-any.whl.metadata
  Using cached gradio-4.7.1-py3-none-any.whl.metadata (17 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/76/ac/dea2939dfc3c591a2494121669455fd7d049248ef284c9542904ddbe05d5/numpy-1.26.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 815.0 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Usin

In [8]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import gradio as gr
import numpy as np

2023-12-04 12:35:51.105091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Build the neural network model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [10]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the model for later use
model.save('digit_recognition_model.h5')

Epoch 1/3
750/750 [==============================] - 25s 31ms/step - loss: 0.1979 - accuracy: 0.9400 - val_loss: 0.0643 - val_accuracy: 0.9818
Epoch 2/3
750/750 [==============================] - 22s 30ms/step - loss: 0.0564 - accuracy: 0.9820 - val_loss: 0.0532 - val_accuracy: 0.9843
Epoch 3/3
313/313 [==============================] - 2s 5ms/step - loss: 0.0438 - accuracy: 0.9872
Test accuracy: 0.9872000217437744


/usr/local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from PIL import Image
# model = load_model('digit_recognition_model.h5')
examples=[
        ("images/example1.png"),
        ("images/example2_1.png"),
        ("images/example2_2.png"),
        ("images/example3.png"),
        ("images/example4.png"),
        ("images/example5.png"),
        ("images/example6_1.png"),
        ("images/example6_2.png"),
        ("images/example9.png"),
    ]

def predict_digit(image):
    img_array = np.array(image)
    img_array = img_array.mean(axis=-1)
    img_array_resized = np.array(Image.fromarray(img_array).resize((img_size, img_size)))
    img_array_resized = img_array_resized.reshape(1, img_size, img_size, 1).astype('float32') / 255.0
    prediction = model.predict(img_array_resized)[0]
    return {label: float(pred) for label, pred in zip(labels, prediction)}

labels = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
img_size = 28
label = gr.Label(num_top_classes=3)

# Create the Gradio interface
iface = gr.Interface(fn=predict_digit, inputs="image", outputs=label, examples=examples)

# Launch the UI
iface.launch()

Running on local URL:  http://127.0.0.1:7929

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 39ms/step
